In [44]:
import pandas as pd

In [45]:
df = pd.read_csv("heart.csv")
print(df.shape)

(918, 12)


In [46]:
df = df[df.RestingBP <= df.RestingBP.mean() + 3 * df.RestingBP.std()]
df = df[df.RestingBP >= df.RestingBP.mean() - 3 * df.RestingBP.std()]
print(df.shape)

(910, 12)


In [47]:
df = df[df.Cholesterol <= df.Cholesterol.mean() + 3 * df.Cholesterol.std()]
print(df.shape)

(907, 12)


In [48]:
df = df[df.MaxHR >= df.MaxHR.mean() - 3 * df.MaxHR.std()]
print(df.shape)

(906, 12)


In [49]:
df = df[df.Oldpeak <= df.Oldpeak.mean() + 3 * df.Oldpeak.std()]
df = df[df.Oldpeak >= df.Oldpeak.mean() - 3 * df.Oldpeak.std()]
print(df.shape)

(899, 12)


In [50]:
df.ExerciseAngina.replace({'N': 0, 'Y': 1}, inplace=True)
df.Sex.replace({'F': 0, 'M': 1}, inplace=True)
df.head()

/var/folders/46/4q6hdhfx7fg5rsk9bzx10cbw0000gn/T/ipykernel_3671/1560280087.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.ExerciseAngina.replace({'N': 0, 'Y': 1}, inplace=True)
/var/folders/46/4q6hdhfx7fg5rsk9bzx10cbw0000gn/T/ipykernel_3671/1560280087.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,ATA,140,289,0,Normal,172,0,0.0,Up,0
1,49,0,NAP,160,180,0,Normal,156,0,1.0,Flat,1
2,37,1,ATA,130,283,0,ST,98,0,0.0,Up,0
3,48,0,ASY,138,214,0,Normal,108,1,1.5,Flat,1
4,54,1,NAP,150,195,0,Normal,122,0,0.0,Up,0


In [51]:
df = pd.get_dummies(df, drop_first=True)
df.head()

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,HeartDisease,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ST_Slope_Flat,ST_Slope_Up
0,40,1,140,289,0,172,0,0.0,0,True,False,False,True,False,False,True
1,49,0,160,180,0,156,0,1.0,1,False,True,False,True,False,True,False
2,37,1,130,283,0,98,0,0.0,0,True,False,False,False,True,False,True
3,48,0,138,214,0,108,1,1.5,1,False,False,False,True,False,True,False
4,54,1,150,195,0,122,0,0.0,0,False,True,False,True,False,False,True


In [52]:
x = df.drop(["HeartDisease"], axis="columns")
y = df.HeartDisease

In [54]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled

array([[-1.42815446,  0.515943  ,  0.46590022, ..., -0.48989795,
        -0.99888827,  1.13469459],
       [-0.47585532, -1.93819859,  1.63471366, ..., -0.48989795,
         1.00111297, -0.88129441],
       [-1.7455875 ,  0.515943  , -0.1185065 , ...,  2.04124145,
        -0.99888827,  1.13469459],
       ...,
       [ 0.3706328 ,  0.515943  , -0.1185065 , ..., -0.48989795,
         1.00111297, -0.88129441],
       [ 0.3706328 , -1.93819859, -0.1185065 , ..., -0.48989795,
         1.00111297, -0.88129441],
       [-1.63977649,  0.515943  ,  0.34901888, ..., -0.48989795,
        -0.99888827,  1.13469459]])

In [56]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

In [58]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
cross_val_score(SVC(), x, y, cv=5).mean()

0.6895779019242706

In [60]:
from sklearn.ensemble import BaggingClassifier
svc_bag = BaggingClassifier(
    estimator=SVC(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
svc_bag.fit(x_train, y_train)
svc_bag.oob_score_

0.8650904033379694

In [62]:
svc_bag.score(x_test, y_test)

0.8944444444444445

In [61]:
from sklearn.tree import DecisionTreeClassifier
cross_val_score(DecisionTreeClassifier(), x, y, cv=5).mean()

0.7195965238981998

In [63]:
from sklearn.ensemble import BaggingClassifier
dtc_bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
dtc_bag.fit(x_train, y_train)
dtc_bag.oob_score_

0.847009735744089

In [65]:
dtc_bag.score(x_test, y_test)

0.8611111111111112